# Spatial Data

Overview of today's topics:

  - Working with shapefiles, GeoPackages, CSV files, and rasters
  - Projection
  - Geometric operations
  - Spatial joins
  - Web mapping
  - Spatial indexing

In [ ]:
import ast
import contextily as cx
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import rasterio.features

## 1. Loading a shapefile or GeoPackage

In [ ]:
# tell geopandas to read a shapefile with its read_file() function, passing in the shapefile folder
# this produces a GeoDataFrame
gdf_tracts = gpd.read_file('../../data/tl_2020_06_tract/')
gdf_tracts.shape

In [ ]:
# just like regular pandas, see the first 5 rows of the GeoDataFrame
# this is a shapefile of polygon geometries, that is, tract boundaries
gdf_tracts.head()

In [ ]:
# rudimentary mapping is as easy as calling the GeoDataFrame's plot method
ax = gdf_tracts.plot()

In [ ]:
# what is the CRS?
# this derives from the shapefile's .prj file
# always make sure the shapefile you load has prj info so you get a CRS attribute!
gdf_tracts.crs

In [ ]:
# loading a GeoPackage works the same way
gdf_stations = gpd.read_file('../../data/rail_stations.gpkg')
gdf_stations.shape

In [ ]:
gdf_stations.crs

## 2. Loading a CSV file

Often, you won't have a shapefile or GeoPackage (which is explicitly spatial), but rather a CSV file which is implicitly spatial (contains lat-lng columns). If you're loading a CSV file (or other non-explicitly spatial file type) of lat-lng data:

  1. first load the CSV file as a DataFrame the usual way with pandas
  2. then create a new geopandas GeoDataFrame from your DataFrame
  3. manually create a geometry column
  4. set the CRS

In [ ]:
# load business location data as a regular pandas dataframe
df = pd.read_csv('../../data/Listing_of_Active_Businesses.csv')
df.shape

In [ ]:
# clean up the data (same code from the data cleaning lecture)
df.columns = df.columns.str.lower().str.replace(' ', '_').str.strip('_#')
df = df.set_index('location_account').sort_index()
df['location_start_date'] = pd.to_datetime(df['location_start_date'])
slicer = pd.IndexSlice[:, 'business_name':'mailing_city']
df.loc[slicer] = df.loc[slicer].apply(lambda col: col.str.title(), axis='rows')
mask = pd.notnull(df['location'])
latlng = df.loc[mask, 'location'].map(ast.literal_eval)
df.loc[mask, ['lat', 'lng']] = pd.DataFrame(latlng.to_list(), index=latlng.index, columns=['lat', 'lng'])
df = df.drop(columns=['location']).dropna(subset=['lat', 'lng'])

In [ ]:
# examine first five rows
df.head()

In [ ]:
# create a geopandas geodataframe from the pandas dataframe
gdf_business = gpd.GeoDataFrame(df)
gdf_business.shape

In [ ]:
# create a geometry column to contain shapely geometry for geopandas to use
# notice the shapely points are lng, lat so that they are equivalent to x, y
# also notice that we set the CRS explicitly
gdf_business['geometry'] = gpd.points_from_xy(x=gdf_business['lng'],
                                              y=gdf_business['lat'])
gdf_business.crs = 'epsg:4326'
gdf_business.shape

**Always define the CRS** if you are manually creating a GeoDataFrame! Earlier, when we loaded the shapefile, geopandas loaded the CRS from the shapefile itself. But our CSV file is not explicitly spatial and it contains no CRS data, so we have to tell it what it is. In our case, the CRS is EPSG:4326, which is WGS84 lat-lng data, such as for GPS. Your data source should always tell you what CRS their coordinates are in. If they don't, ask! Don't just guess.

In [ ]:
gdf_business.head()

In [ ]:
# what's the CRS
gdf_business.crs

## 3. Loading a raster

So far we've worked with vector data. We can also work with raster data. Raster datasets are grids of pixels, where each pixel has a value (or multiple values if multiple bands of data), while vector datasets contain geometry objects with attributes, where each geometry is represented by mathematical coordinates for points, lines, polygons, etc. "Raster is faster but vector is corrector."

Common raster data include:
  - tree cover
  - urbanization footprints
  - land use
  - elevation

In this example we load the SRTM 30m elevation raster, downloaded from https://dwtkns.com/srtm30m/, and cropped (to make a small dataset that can fit in laptop memory) via [raster-crop-bbox.ipynb](raster-crop-bbox.ipynb)

In [ ]:
# load the raster file and view its band count, pixel width and height, null value, and geographic bounds
raster = rasterio.open('../../data/la-elevation.tif')
print(raster.count, raster.width, raster.height)
print(raster.nodata)
print(raster.bounds)

In [ ]:
# view the raster data
df = pd.DataFrame(raster.read(1))
df

In [ ]:
# histogram of elevations (meters above sea level) around downtown LA
ax = df[df!=raster.nodata].stack().hist(bins=50)

In [ ]:
# get shapes representing groups of adjacent pixels with same values
# affine transformation maps pixel row/col -> spatial x/y
shapes = rasterio.features.shapes(source=raster.read(1),
                                  transform=raster.transform)

In [ ]:
# convert raster to GeoJSON-like vector features and create a gdf from them
# pro-tip: use generator comprehension for memory efficiency
features = ({'geometry': polygon, 'properties': {'elevation': value}} for polygon, value in shapes)
gdf_srtm = gpd.GeoDataFrame.from_features(features, crs=raster.crs)

# drop any null rows
gdf_srtm = gdf_srtm[gdf_srtm['elevation']!=raster.nodata]
gdf_srtm.shape

In [ ]:
# view the gdf
gdf_srtm

In [ ]:
# check its crs
gdf_srtm.crs

In [ ]:
# plot the elevation pixels and identify pershing square
fig, ax = plt.subplots(facecolor='#111111')
ax = gdf_srtm.plot(ax=ax, column='elevation', cmap='inferno')
_ = ax.axis('off')
_ = ax.scatter(y=34.048097, x=-118.253233, c='w', marker='x', s=100)

In [ ]:
# now it's your turn
# change the colors and also show the location of city hall on the map


## 4. Projection

Your datasets need to be in the same CRS if you want to work with them together. If they're not, then project one or more of them so they're in the same CRS.

Take note of the important difference here between *setting* a CRS (i.e., identifying a dataset's current CRS) and *projecting* to a CRS (i.e., mathematically transforming your coordinates from their current CRS to a different one). Projection lets you transform, for example, from lat-lng coordinates on the surface of the round Earth to a flat two-dimensional plane for mapping and analysis in intuitive units like meters.

In [ ]:
# check if all our datasets have the same CRS
gdf_tracts.crs == gdf_stations.crs == gdf_business.crs == gdf_srtm.crs

In [ ]:
# project them all to UTM zone 11N (see http://epsg.io/32611)
utm_crs = 'epsg:32611'
gdf_tracts = gdf_tracts.to_crs(utm_crs)
gdf_stations = gdf_stations.to_crs(utm_crs)
gdf_business = gdf_business.to_crs(utm_crs)
gdf_srtm = gdf_srtm.to_crs(utm_crs)

In [ ]:
# check if all our datasets have the same CRS
gdf_tracts.crs == gdf_stations.crs == gdf_business.crs == gdf_srtm.crs

**Be careful**: heed the difference between the `gdf.crs` attribute and the `gdf.to_crs()` method. The former is the geodataframe's current CRS, whereas the latter projects the geodataframe to a new CRS.

In [ ]:
# now it's your turn
# pick a different CRS and re-project the data to it


## 5. Geometric operations

GIS and spatial analysis use common "computational geometry" operations like intersects, within, and dissolve.

In [ ]:
%%time
# takes a few seconds...
# dissolve lets you aggregate (merge geometries together) by shared attribute values
# this is the spatial equivalent of pandas's groupby function
gdf_counties = gdf_tracts.dissolve(by='COUNTYFP', aggfunc=np.sum)

In [ ]:
# now that we've dissolved tracts -> counties and summed their attributes,
# plot the counties by land area
fig, ax = plt.subplots(facecolor='#111111')
ax = gdf_counties.plot(ax=ax, column='ALAND', cmap='Blues_r')
_ = ax.axis('off')

In [ ]:
# just like in regular pandas, we can filter and subset the GeoDataFrame
# retain only tracts in LA county (FIPS code 037)
mask = gdf_tracts['COUNTYFP'] == '037'
gdf_tracts_la = gdf_tracts[mask]
ax = gdf_tracts_la.plot()

In [ ]:
# discard the channel islands' tracts to retain only the mainland
# how? sort by centroids' y-coord and discard the two southern-most
labels = gdf_tracts_la.centroid.y.sort_values().iloc[2:].index
gdf_tracts_la = gdf_tracts_la.loc[labels]
ax = gdf_tracts_la.plot()

In [ ]:
# unary union merges all geometries in gdf into one
la_geom = gdf_tracts_la.unary_union
la_geom

In [ ]:
# convex hull generates the minimal convex polygon around feature(s)
la_geom.convex_hull

In [ ]:
# envelope generates the minimal rectangular polygon around feature(s)
la_geom.envelope

In [ ]:
# get a bounding box around our elevation data
elev_bounds = gdf_srtm.unary_union.envelope

Many spatial operations, such as intersects/within, scale in time complexity as a function of 1) the number of objects, and 2) the number of vertices in the reference polygon. Using a simplified reference polygon, such as a bounding box, can drastically speed up your operation at the cost of imprecision. In this case, our raster is already approximately square, and we don't need to do precise matching, so let's use the bounding box for intersects/within to filter our tracts and businesses to those that lie within the area covered by our elevation data.

In [ ]:
# get all the tracts that intersect those bounds
# intersects tells you if each geometry in one dataset intersects with some other (single) geometry
mask = gdf_tracts_la.intersects(elev_bounds)
gdf_tracts_dtla = gdf_tracts_la[mask]
gdf_tracts_dtla.shape

In [ ]:
# get all business points within those bounds
# within tells you if each geometry in one dataset is within some other (single) geometry
mask = gdf_business.within(elev_bounds)
gdf_business_dtla = gdf_business[mask]
gdf_business_dtla.shape

In [ ]:
# euclidean buffers let you analyze the area around features (use projected CRS!)
# buffer the rail stations by a half km (5-10 minute walk)
gdf_stations['geometry'] = gdf_stations.buffer(500)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8), facecolor='#111111')
ax = gdf_tracts_dtla.plot(ax=ax, color='k')
ax = gdf_stations.plot(ax=ax, color='w', alpha=0.3)
ax = gdf_business_dtla.plot(ax=ax, color='#ffff66', marker='.', linewidth=0, markersize=20, alpha=0.05)
_ = ax.axis('off')

In [ ]:
# you can do set operations like union, intersection, and difference
# get all the portions of tracts >0.5km from a rail station
gdf_diff = gpd.overlay(gdf_tracts_dtla, gdf_stations, how='difference')
ax = gdf_diff.plot()

## 6. Spatial join

Joins two geodataframes based on some shared spatial location.

In [ ]:
# remember (again): always double-check CRS before any spatial operations
gdf_tracts_dtla.crs == gdf_stations.crs == gdf_business.crs == gdf_srtm.crs

### 6a. How hilly is it around the stations?

In [ ]:
# join stations to elevation data
gdf = gpd.sjoin(gdf_srtm, gdf_stations, how='inner', predicate='intersects')

In [ ]:
# counts vary because these aren't elevation pixels, but regions of same value
gdf_elev_desc = gdf.groupby('name')['elevation'].describe().astype(int)
gdf_elev_desc

In [ ]:
gdf_stations_elev = gdf_stations.merge(gdf_elev_desc, left_on='name', right_index=True)
gdf_stations_elev.head()

In [ ]:
# which stations have the greatest elevation variation around them?
ax = gdf_stations_elev.plot(column='std')

In [ ]:
# now it's your turn
# which station buffer covers the largest elevation range?


### 6b. Which stations have the most businesses in their catchment areas?

In [ ]:
# join stations to businesses data
gdf = gpd.sjoin(gdf_business, gdf_stations, how='inner', predicate='intersects')

In [ ]:
# counts vary because these aren't elevation pixels, but regions of same value
gdf_business_desc = gdf.groupby('name').size().sort_values(ascending=False)
gdf_business_desc.name = 'count'
gdf_business_desc

Beware artificial peripheries! Some station buffers extend beyond the spatially-cropped business locations. How would you fix this?

In [ ]:
# now it's your turn
# change earlier parts of the notebook to make sure our station buffers capture all the businesses within them


In [ ]:
gdf_stations_business = gdf_stations.merge(gdf_business_desc, left_on='name', right_index=True)

In [ ]:
# which stations have the most businesses around them?
ax = gdf_stations_business.plot(column='count')

This works ok as a quick and dirty way to visually inspect our results. But it only works because we're analyzing/visualizing counts across study sites (i.e., station buffers) that *are all the same size* as each other. If the study site sizes varied (such as tracts or counties), counts might be correlated with area! Then you're just visualizing which study sites are the largest. In such cases, make sure you normalize. For example, use densities instead of counts.

### 6c. Which tracts have the most businesses?

In [ ]:
# join tracts to business data
gdf = gpd.sjoin(gdf_business, gdf_tracts_dtla, how='inner', predicate='intersects')

In [ ]:
# count businesses per tract
counts = gdf.groupby('GEOID').size()
counts.name = 'count'

In [ ]:
# merge in the counts then calculate density (businesses per km^2)
gdf_tracts_dtla_business = gdf_tracts_dtla.merge(counts, left_on='GEOID', right_index=True)
gdf_tracts_dtla_business['density'] = gdf_tracts_dtla_business['count'] / gdf_tracts_dtla_business['ALAND'] * 1e6

In [ ]:
# plot tracts as choropleth plus station buffers
fig, ax = plt.subplots(figsize=(8, 8), facecolor='#111111')
ax = gdf_tracts_dtla_business.plot(ax=ax, column='density', cmap='viridis')
ax = gdf_stations.plot(ax=ax, alpha=0.2, linewidth=3, edgecolor='w', color='none')
_ = ax.axis('off')

In [ ]:
# this time, let's add a basemap for context
fig, ax = plt.subplots(figsize=(8, 8), facecolor='#111111')
ax = gdf_tracts_dtla_business.plot(ax=ax, column='density', cmap='viridis',
                                   alpha=0.7, linewidth=0.3, edgecolor='k')
ax = gdf_stations.plot(ax=ax, alpha=0.3, linewidth=3, edgecolor='w', color='none')
_ = ax.axis('off')

# add the basemap with contextily, choosing a tile provider
# or try cx.providers.Stamen.TonerBackground, etc
cx.add_basemap(ax, crs=gdf_stations.crs.to_string(),
               source=cx.providers.CartoDB.DarkMatter)

ax.figure.savefig('map.png', dpi=600, bbox_inches='tight')

In [ ]:
# now it's your turn
# change the tile provider, the tract colors, the alphas, etc to find a plot your like


How about an interactive web map instead?

In [ ]:
# optionally bin the data into quintiles
bins = list(gdf_tracts_dtla_business['density'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))

# create leaflet choropleth web map
m = folium.Map(location=(34.047223, -118.253555), zoom_start=15, tiles='cartodbdark_matter')
c = folium.Choropleth(geo_data=gdf_tracts_dtla_business,
                      data=gdf_tracts_dtla_business,
                      #bins=bins,
                      columns=['GEOID', 'density'],
                      key_on='feature.properties.GEOID', 
                      highlight=True,
                      fill_color='YlOrRd_r', 
                      legend_name='Businesses per square km').add_to(m)

# add mouseover tooltip to the countries
c.geojson.add_child(folium.features.GeoJsonTooltip(['GEOID', 'density']))

# save web map to disk
m.save('webmap.html')

In [ ]:
m

In [ ]:
# now it's your turn
# try binning the data in different ways. how would you do it?
# try changing the colors, basemap, and what variable you're visualizing


## 7. Spatial Indexing

When you need to find which page a topic appears on in a book, do you search through every word, page by page, until you find it? When you need to find which polygon a point lies in, do you search through every polygon, one at a time, until you find it? Sometimes. But you can avoid that slow brute-force search if you use an index.

A spatial index such as an [r-tree](https://en.wikipedia.org/wiki/R-tree) can drastically speed up spatial operations like intersects and joins. In computer science, a tree data structure represents parent and children objects like the branches of a tree. For example, a k-d tree lets you partition space for fast nearest-neighbor search. But an r-tree is particularly useful for finding what geometries intersect with some other geometry, such as point-in-polygon queries.

An r-tree represents individual objects and their bounding boxes ("r" is for "rectangle") as the lowest level of the spatial index. It then aggregates nearby objects and represents them with their aggregate bounding box in the next higher level of the index. At yet higher levels, the r-tree aggregates bounding boxes and represents them by their bounding box, iteratively, until everything is nested into one top-level bounding box.

To search, the r-tree takes a query box and, starting at the top level, sees which (if any) bounding boxes intersect it. It then expands each intersecting bounding box and sees which of the child bounding boxes inside it intersect the query box. This proceeds recursively until all intersecting boxes are searched down to the lowest level, and returns the matching objects from the lowest level.

In [ ]:
# geopandas uses r-tree spatial indexes
# if a spatial index doesn't already exist,
# it will be created the first time the sindex attribute is accessed
sindex = gdf_business.sindex

In [ ]:
%%time
# count all the businesses within the station buffers
polygon = gdf_stations.unary_union
gdf_business.within(polygon).sum()

We can break this out into a two-step process. First find approximate matches with spatial index, then precise matches from those approximate ones.

In [ ]:
len(gdf_business)

In [ ]:
%%time
# get the positions of possible matches, to use with iloc
positions = sindex.intersection(polygon.bounds)
possible_matches = gdf_business.iloc[positions]
len(possible_matches)

That was fast! And we're nearly there. We intersected the spatial index with the bounds of our polygon. This returns a set of possible matches. That is, there are no false negatives but there may be some false positives if an r-tree rectangle within the bounds contains some points outside the tracts' true borders.

Unfortunately, the heavy lifting remains in filtering down the possible matches within the bounds to figure out which are within the polygon itself. To identify the precise matches (those points exactly within our polygon), we intersect the possible matches with the polygon itself.

In [ ]:
%%time
mask = possible_matches.intersects(polygon)
precise_matches = possible_matches[mask]
len(precise_matches)

So, the r-tree lets us filter out ~90% of the points (from the rest of the county) nearly instantly, but then the final precise point-in-polygon search (i.e., of the remaining points within the station buffers' bounding box, which are within the station buffers themselves?) consumes nearly all the runtime.

In [ ]:
# what false positives appeared among the possible matches?
labels = possible_matches.index.difference(precise_matches.index)
false_positives = possible_matches.loc[labels]

In [ ]:
# visualize the precise matches vs the false positives
ax = gdf_stations.plot(color='gray')
ax = false_positives.plot(ax=ax, c='r', markersize=0.1)
ax = precise_matches.plot(ax=ax, c='b', markersize=0.1)
_ = ax.axis('off')